In [ ]:
pip install -q bitsandbytes accelerate transformers datasets peft torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Load Mistral-7B-Instruct in 4-bit

In [ ]:
from huggingface_hub import login
from google.colab import userdata

# Fetch your Hugging Face token from Colab secrets
hf_token = userdata.get("HF_TOKEN")

# Log in to Hugging Face
login(token=hf_token)

Tokenize Prompt–Response Pairs for Causal LM

In [ ]:
from transformers import BitsAndBytesConfig
import torch
from transformers import AutoModelForCausalLM

# Define quantization configuration for 4-bit loading
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,              # enable 4-bit quantization
    bnb_4bit_use_double_quant=True, # use nested quantization
    bnb_4bit_compute_dtype=torch.float16,  # compute dtype
    bnb_4bit_quant_type="nf4"       # quantization type
)

# Reload model with quantization config
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",  # model name
    quantization_config=quant_config,     # updated quantization method
    device_map="auto"                     # place model layers on available GPU
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

# Enhanced LoRA configuration for A100 (40GB)
peft_config = LoraConfig(
    r=16,                         # increase LoRA rank for better expressiveness
    lora_alpha=32,               # scale factor adjusted for higher rank
    target_modules=["q_proj", "v_proj"],  # attention projection layers
    lora_dropout=0.05,           # regularization
    bias="all",                  # train bias parameters as well
    task_type=TaskType.CAUSAL_LM # task type for language modeling
)

# Attach LoRA to base model
model = get_peft_model(model, peft_config)

# Print updated trainable parameter count
model.print_trainable_parameters()

trainable params: 6,815,744 || all params: 7,248,547,840 || trainable%: 0.0940


Load Dataset and Tokenizer

In [ ]:
import json
from transformers import AutoTokenizer

# Load formatted dataset from Drive
with open("/content/drive/Shareddrives/DATA266_Project/Data/spider_data/spider_instruction_data.json", "r") as f:
    dataset = json.load(f)

# Reload tokenizer for Mistral
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

 Set pad_token for Mistral Tokenizer

In [ ]:
# Set the pad_token to eos_token (common fix for decoder-only models)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from transformers import PreTrainedTokenizerBase

# Tokenize each example into input_ids and labels
def tokenize_sample(sample):
    # Combine prompt and response for causal LM
    full_prompt = sample["prompt"] + "\n" + sample["response"]

    # Tokenize the entire string
    tokenized = tokenizer(
        full_prompt,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )

    # Labels are the same as input_ids for causal LM
    tokenized["labels"] = tokenized["input_ids"].clone()
    return {key: val.squeeze() for key, val in tokenized.items()}

# Apply tokenizer to dataset
tokenized_dataset = [tokenize_sample(example) for example in dataset]

 Wrap Tokenized Data in a Dataset

In [ ]:
from torch.utils.data import Dataset

# Define a simple custom dataset class
class PromptDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            "input_ids": self.data[idx]["input_ids"],
            "attention_mask": self.data[idx]["attention_mask"],
            "labels": self.data[idx]["labels"]
        }

# Create dataset instance
train_dataset = PromptDataset(tokenized_dataset)

Set Up Training Arguments and Trainer

In [ ]:
from transformers import TrainingArguments, Trainer
from torch.utils.data import DataLoader
import os

# Define training output directory
output_dir = "/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_spider_lora"

# Define training arguments
training_args = TrainingArguments(
    output_dir=output_dir,             # directory to save checkpoints
    per_device_train_batch_size=4,     # batch size per device
    gradient_accumulation_steps=4,     # effective batch size = 4 x 4 = 16
    num_train_epochs=3,                # number of epochs
    learning_rate=2e-4,                # learning rate
    fp16=True,                         # use float16 training
    logging_steps=10,                  # log every 10 steps
    save_strategy="epoch",             # save at each epoch
    report_to="none"                   # no W&B or TensorBoard
)

 Initialize Trainer and Start Training

In [ ]:
from transformers import Trainer

# Initialize Trainer with model, training args, and dataset
trainer = Trainer(
    model=model,                         # LoRA-wrapped Mistral model
    args=training_args,                  # training configuration
    train_dataset=train_dataset,         # our prompt-tokenized dataset
)

# Start training
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,0.658000
20,0.474600
30,0.392300
40,0.355100
50,0.305800
60,0.273800
70,0.244400
80,0.211100
90,0.205100
100,0.164000


TrainOutput(global_step=1311, training_loss=0.10129983225321607, metrics={'train_runtime': 2464.4129, 'train_samples_per_second': 8.521, 'train_steps_per_second': 0.532, 'total_flos': 4.582880059588608e+17, 'train_loss': 0.10129983225321607, 'epoch': 2.994285714285714})

In [ ]:
from transformers import AutoModelForCausalLM
import torch
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    torch_dtype=torch.float16  # or .bfloat16 based on your runtime
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer.pad_token = tokenizer.eos_token  # required for generation

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from peft import PeftModel

# Load your fine-tuned adapter into the base model
peft_model = PeftModel.from_pretrained(
    base_model,
    "/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_spider_lora/checkpoint-1311"
)
# Merge LoRA into base model
merged_model = peft_model.merge_and_unload()

In [ ]:
merged_model.save_pretrained("/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_merged_full")
tokenizer.save_pretrained("/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_merged_full")

('/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_merged_full/tokenizer_config.json',
 '/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_merged_full/special_tokens_map.json',
 '/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_merged_full/tokenizer.model',
 '/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_merged_full/added_tokens.json',
 '/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_merged_full/tokenizer.json')

In [ ]:
from peft import PeftModel

# Load your fine-tuned adapter into the base model
peft_model = PeftModel.from_pretrained(
    base_model,
    "/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_complex_sql_training_4090/checkpoint-14500"
)
# Merge LoRA into base model
merged_model = peft_model.merge_and_unload()

In [ ]:
merged_model.save_pretrained("/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_merged_full_4090")
tokenizer.save_pretrained("/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_merged_full_4090")

('/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_merged_full_4090/tokenizer_config.json',
 '/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_merged_full_4090/special_tokens_map.json',
 '/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_merged_full_4090/tokenizer.model',
 '/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_merged_full_4090/added_tokens.json',
 '/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_merged_full_4090/tokenizer.json')

## seeklhy/SynSQL-2.5M - Fine tuning

In [ ]:
import torch
import json
import os
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from torch.utils.data import Dataset
from huggingface_hub import login
from google.colab import userdata
import re
import random
from tqdm.auto import tqdm

# Set random seed for reproducibility
torch.manual_seed(42)
random.seed(42)

# Fetch your Hugging Face token from Colab secrets (if needed)
hf_token = userdata.get("HF_TOKEN")
if hf_token:
    login(token=hf_token)

In [ ]:
def is_complex_query(sql_query):
    #Determine if a SQL query is complex based on specific patterns.
    sql_lower = sql_query.lower()

    # Check for multiple joins
    join_count = sql_lower.count(" join ")

    # Check for subqueries
    has_subquery = "select" in sql_lower[sql_lower.find("from"):] if "from" in sql_lower else False

    # Check for complex functions and operations
    complex_patterns = [
        "group by", "having", "order by limit", "distinct",
        "case when", "with", "union", "intersect", "except",
        "sum(", "avg(", "max(", "min(", "count("
    ]
    has_complex_funcs = any(pattern in sql_lower for pattern in complex_patterns)

    # Check for multiple conditions
    condition_count = sql_lower.count(" and ") + sql_lower.count(" or ")

    # Count the number of tables involved (approximate)
    from_parts = sql_lower.split("from")[1:]
    table_references = 0
    for part in from_parts:
        if "where" in part:
            part = part.split("where")[0]
        elif "group by" in part:
            part = part.split("group by")[0]
        table_references += part.count(",") + 1  # Count commas + 1 for table references

    # Define complexity threshold
    is_complex = (join_count >= 1) or has_subquery or (condition_count >= 2) or \
                 has_complex_funcs or (table_references >= 2)

    return is_complex

In [ ]:
import os
import json
import random
import numpy as np
from tqdm import tqdm
from datasets import load_dataset

# Load dataset in streaming mode to save memory
print("Loading SynSQL dataset...")
synsql_stream = load_dataset("seeklhy/SynSQL-2.5M", split="train", streaming=True)

# Set target numbers
target_complex = 35000  # Target number of complex queries
target_simple = 25000   # Target number of simple queries
final_dataset_size = target_complex + target_simple

# Create output directory
os.makedirs("sql_data", exist_ok=True)

# Files for storing complex and simple queries
complex_file = "sql_data/complex_queries.jsonl"
simple_file = "sql_data/simple_queries.jsonl"

# Remove files if they exist
if os.path.exists(complex_file):
    os.remove(complex_file)
if os.path.exists(simple_file):
    os.remove(simple_file)

# Tracking variables
complex_count = 0
simple_count = 0
processed_count = 0

# Track progress
progress_bar = tqdm(desc="Processing examples")

# Create an iterator for the stream
synsql_iterator = iter(synsql_stream)

print(f"Collecting {target_complex} complex and {target_simple} simple examples...")

# Process the stream and write directly to files
while complex_count < target_complex or simple_count < target_simple:
    try:
        sample = next(synsql_iterator)
        processed_count += 1

        question = sample.get("question", "").strip()
        sql = sample.get("sql", "").strip()

        if not question or not sql:
            continue

        prompt = f"""### Instruction:
Generate an SQL query for the following question.
### Question:
{question}
### SQL:
"""
        formatted_example = {
            "prompt": prompt,
            "response": sql
        }

        # Write to appropriate file (JSONL format - one JSON object per line)
        if is_complex_query(sql) and complex_count < target_complex:
            with open(complex_file, 'a') as f:
                f.write(json.dumps(formatted_example) + '\n')
            complex_count += 1

        elif not is_complex_query(sql) and simple_count < target_simple:
            with open(simple_file, 'a') as f:
                f.write(json.dumps(formatted_example) + '\n')
            simple_count += 1

        # Update progress every 1000 examples
        if processed_count % 1000 == 0:
            progress_bar.update(1000)
            print(f"\nFound {complex_count}/{target_complex} complex queries and {simple_count}/{target_simple} simple queries")

        # Safety check
        if processed_count > 1000000:
            print("Warning: Processed too many examples without finding enough of both types.")
            break

    except StopIteration:
        print("\nReached the end of the dataset before finding enough examples.")
        break

progress_bar.close()

print("\nCreating high-quality randomized dataset...")

# Create index arrays for both files
complex_indices = list(range(min(complex_count, target_complex)))
simple_indices = list(range(min(simple_count, target_simple)))

# Shuffle the indices
random.shuffle(complex_indices)
random.shuffle(simple_indices)

# Calculate how many examples we can include from each type
actual_complex = min(complex_count, target_complex)
actual_simple = min(simple_count, target_simple)
total_examples = actual_complex + actual_simple

# Determine indices for interleaving complex and simple examples
# This creates a proper randomized dataset while respecting the target ratio
all_indices = []
for i in range(total_examples):
    # Weighted random selection based on remaining examples
    complex_weight = actual_complex / (actual_complex + actual_simple) if (actual_complex + actual_simple) > 0 else 0
    if random.random() < complex_weight and complex_indices:
        all_indices.append(("complex", complex_indices.pop()))
        actual_complex -= 1
    elif simple_indices:
        all_indices.append(("simple", simple_indices.pop()))
        actual_simple -= 1
    else:
        all_indices.append(("complex", complex_indices.pop()))
        actual_complex -= 1

# Function to get example at a specific index from a file
def get_example_at_index(file_path, index):
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            if i == index:
                return json.loads(line)
    return None

# Save the interleaved dataset in chunks
chunk_size = 10000
total_chunks = (total_examples - 1) // chunk_size + 1
print(f"Saving properly randomized dataset in {total_chunks} chunks...")

for chunk_idx in range(total_chunks):
    start_idx = chunk_idx * chunk_size
    end_idx = min(start_idx + chunk_size, total_examples)

    chunk = []
    for i in range(start_idx, end_idx):
        file_type, file_idx = all_indices[i]
        if file_type == "complex":
            example = get_example_at_index(complex_file, file_idx)
        else:
            example = get_example_at_index(simple_file, file_idx)
        if example:
            chunk.append(example)

    chunk_file = f"sql_data/training_data_chunk_{chunk_idx}.json"
    with open(chunk_file, 'w') as f:
        json.dump(chunk, f)
    print(f"Saved chunk {chunk_idx} with {len(chunk)} examples")

# Create a manifest file for training
manifest = {
    "total_examples": total_examples,
    "complex_examples": min(complex_count, target_complex),
    "simple_examples": min(simple_count, target_simple),
    "chunks": total_chunks,
    "chunk_size": chunk_size
}

with open("sql_data/manifest.json", 'w') as f:
    json.dump(manifest, f)

print(f"Dataset creation complete. Manifest saved to sql_data/manifest.json")
print(f"Final dataset composition:")
print(f" - Complex queries: {min(complex_count, target_complex)}")
print(f" - Simple queries added: {min(simple_count, target_simple)}")
print(f" - Total training examples: {total_examples}")

Loading SynSQL dataset...


Processing examples: 0it [00:00, ?it/s]

Processing examples: 3000it [03:33, 18.03it/s]


Found 927/35000 complex queries and 73/25000 simple queries

Found 1869/35000 complex queries and 131/25000 simple queries

Found 2832/35000 complex queries and 168/25000 simple queries

Found 3773/35000 complex queries and 227/25000 simple queries


Processing examples: 7000it [03:34, 64.19it/s]

...


Found 35000/35000 complex queries and 24512/25000 simple queries

Found 35000/35000 complex queries and 24563/25000 simple queries

Found 35000/35000 complex queries and 24626/25000 simple queries

Found 35000/35000 complex queries and 24678/25000 simple queries

Found 35000/35000 complex queries and 24705/25000 simple queries

Found 35000/35000 complex queries and 24723/25000 simple queries


Processing examples: 388000it [03:49, 25749.48it/s]


Found 35000/35000 complex queries and 24734/25000 simple queries

Found 35000/35000 complex queries and 24771/25000 simple queries

Found 35000/35000 complex queries and 24805/25000 simple queries

Found 35000/35000 complex queries and 24824/25000 simple queries

Found 35000/35000 complex queries and 24848/25000 simple queries

Found 35000/35000 complex queries and 24865/25000 simple queries


Processing examples: 394000it [03:49, 1717.11it/s] 



Found 35000/35000 complex queries and 24895/25000 simple queries

Found 35000/35000 complex queries and 24926/25000 simple queries

Found 35000/35000 complex queries and 24956/25000 simple queries

Found 35000/35000 complex queries and 24999/25000 simple queries

Creating high-quality randomized dataset...
Saving properly randomized dataset in 6 chunks...
Saved chunk 0 with 10000 examples
Saved chunk 1 with 10000 examples
Saved chunk 2 with 10000 examples
Saved chunk 3 with 10000 examples
Saved chunk 4 with 10000 examples
Saved chunk 5 with 10000 examples
Dataset creation complete. Manifest saved to sql_data/manifest.json
Final dataset composition:
 - Complex queries: 35000
 - Simple queries added: 25000
 - Total training examples: 60000


In [ ]:
import os
import json
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer
from tqdm import tqdm

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding works correctly

# Define the correct data directory
data_dir = "/content/drive/Shareddrives/DATA266_Project/Data/sql_data"

# Define a dataset class that loads data from chunks
class ChunkedSQLDataset(Dataset):
    def __init__(self, data_dir, tokenizer, max_length=512):
        self.data_dir = data_dir
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Load the manifest file
        with open(os.path.join(data_dir, "manifest.json"), 'r') as f:
            self.manifest = json.load(f)

        self.total_examples = self.manifest["total_examples"]
        self.chunks = self.manifest["chunks"]
        self.chunk_size = self.manifest["chunk_size"]

        # Create a mapping of example index to chunk index and position
        self.example_map = {}
        for i in range(self.total_examples):
            chunk_idx = i // self.chunk_size
            position_in_chunk = i % self.chunk_size
            self.example_map[i] = (chunk_idx, position_in_chunk)

        # Cache for loaded chunks to improve efficiency
        self.chunk_cache = {}
        self.current_cached_chunks = []
        self.max_cached_chunks = 2  # Adjust based on memory constraints

    def __len__(self):
        return self.total_examples

    def load_chunk(self, chunk_idx):
        """Load a chunk into memory if not already cached"""
        if chunk_idx in self.chunk_cache:
            return self.chunk_cache[chunk_idx]

        # Load the chunk
        chunk_file = os.path.join(self.data_dir, f"training_data_chunk_{chunk_idx}.json")
        with open(chunk_file, 'r') as f:
            chunk_data = json.load(f)

        # Update cache
        self.chunk_cache[chunk_idx] = chunk_data
        self.current_cached_chunks.append(chunk_idx)

        # If cache is full, remove the oldest chunk
        if len(self.current_cached_chunks) > self.max_cached_chunks:
            oldest_chunk = self.current_cached_chunks.pop(0)
            if oldest_chunk in self.chunk_cache:
                del self.chunk_cache[oldest_chunk]

        return chunk_data

    def __getitem__(self, idx):
        # Get the chunk index and position
        chunk_idx, position_in_chunk = self.example_map[idx]

        # Load the chunk if necessary
        chunk_data = self.load_chunk(chunk_idx)

        # Get the example
        sample = chunk_data[position_in_chunk]

        # Combine prompt and response
        full_text = sample["prompt"] + sample["response"]

        # Tokenize with proper padding and truncation
        tokenized = self.tokenizer(
            full_text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )

        # For causal LM, labels are the same as input_ids
        tokenized["labels"] = tokenized["input_ids"].clone()

        return {key: val.squeeze(0) for key, val in tokenized.items()}

# Create the dataset
print("Setting up dataset...")
full_dataset = ChunkedSQLDataset(data_dir=data_dir, tokenizer=tokenizer)

# Create train/val/test split using indices
dataset_size = len(full_dataset)
test_size = min(1000, int(dataset_size * 0.05))
val_size = min(1000, int(dataset_size * 0.05))
train_size = dataset_size - val_size - test_size

# Use PyTorch's random_split to create the splits
train_dataset, val_dataset, test_dataset = random_split(
    full_dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)  # For reproducibility
)

print(f"Total dataset size: {dataset_size}")
print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Output directory is the same as input directory
output_dir = data_dir

# Function to save dataset splits
def save_tokenized_dataset(dataset, split_name, output_dir, chunk_size=1000):
    indices = dataset.indices if hasattr(dataset, 'indices') else range(len(dataset))
    num_chunks = (len(indices) - 1) // chunk_size + 1

    print(f"Saving {split_name} dataset ({len(indices)} examples) in {num_chunks} chunks...")

    for chunk_idx in tqdm(range(num_chunks)):
        start_idx = chunk_idx * chunk_size
        end_idx = min(start_idx + chunk_size, len(indices))
        chunk_indices = indices[start_idx:end_idx]

        tokenized_examples = []
        for idx in chunk_indices:
            example = full_dataset[idx]
            tokenized_examples.append(example)

        # Save the chunk
        output_file = os.path.join(output_dir, f"{split_name}_chunk_{chunk_idx}.pt")
        torch.save(tokenized_examples, output_file)

    # Save manifest for this split
    manifest_file = os.path.join(output_dir, f"{split_name}_manifest.json")
    with open(manifest_file, 'w') as f:
        json.dump({
            "num_examples": len(indices),
            "num_chunks": num_chunks,
            "chunk_size": chunk_size
        }, f)

    print(f"Saved {split_name} dataset to {output_dir}")

# Save each split
save_tokenized_dataset(train_dataset, "train", output_dir)
save_tokenized_dataset(val_dataset, "val", output_dir)
save_tokenized_dataset(test_dataset, "test", output_dir)

print("All datasets saved successfully.")

Setting up dataset...
Total dataset size: 60000
Training dataset size: 58000
Validation dataset size: 1000
Test dataset size: 1000
Saving train dataset (58000 examples) in 58 chunks...


100%|██████████| 58/58 [36:55<00:00, 38.21s/it]


Saved train dataset to /content/drive/Shareddrives/DATA266_Project/Data/sql_data
Saving val dataset (1000 examples) in 1 chunks...


100%|██████████| 1/1 [00:39<00:00, 39.53s/it]


Saved val dataset to /content/drive/Shareddrives/DATA266_Project/Data/sql_data
Saving test dataset (1000 examples) in 1 chunks...


100%|██████████| 1/1 [00:39<00:00, 39.11s/it]

Saved test dataset to /content/drive/Shareddrives/DATA266_Project/Data/sql_data
All datasets saved successfully.


In [ ]:
import os
import torch
import json
from tqdm import tqdm

# Define paths
data_dir = "/content/drive/Shareddrives/DATA266_Project/Data/sql_data"
preprocessed_data_path = "/content/drive/Shareddrives/DATA266_Project/Data/sql_data/preprocessed_data"

# Create preprocessed data directory if it doesn't exist
os.makedirs(preprocessed_data_path, exist_ok=True)

# Function to load all tokenized examples from chunks
def load_all_tokenized_data(data_dir, split_names=["train", "val", "test"]):
   print("Loading all tokenized data...")
   all_examples = []
   splits_indices = {"train_indices": [], "val_indices": [], "test_indices": []}

   current_index = 0

   # Process each split
   for split_name in split_names:
       # Load the manifest for this split
       manifest_path = os.path.join(data_dir, f"{split_name}_manifest.json")
       if not os.path.exists(manifest_path):
           print(f"Warning: Manifest not found for {split_name} split")
           continue

       with open(manifest_path, "r") as f:
           manifest = json.load(f)

       num_chunks = manifest["num_chunks"]
       split_start_index = current_index

       # Load all chunks for this split
       for chunk_idx in tqdm(range(num_chunks), desc=f"Loading {split_name} chunks"):
           chunk_path = os.path.join(data_dir, f"{split_name}_chunk_{chunk_idx}.pt")
           if not os.path.exists(chunk_path):
               print(f"Warning: Chunk file not found: {chunk_path}")
               continue

           # Load the chunk
           chunk_data = torch.load(chunk_path)

           # Add current indices to the appropriate split
           chunk_indices = list(range(current_index, current_index + len(chunk_data)))
           splits_indices[f"{split_name}_indices"].extend(chunk_indices)

           # Add examples to the combined list
           all_examples.extend(chunk_data)
           current_index += len(chunk_data)

       print(f"Loaded {current_index - split_start_index} examples for {split_name} split")

   print(f"Total loaded examples: {len(all_examples)}")
   return all_examples, splits_indices

# Load all tokenized data and track indices for each split
tokenized_data, splits_indices = load_all_tokenized_data(data_dir)

# Save the combined tokenized data
combined_file = os.path.join(preprocessed_data_path, "tokenized_combined_data.pt")
print(f"Saving combined tokenized data to {combined_file}")
torch.save(tokenized_data, combined_file)

# Save the split indices
splits_file = os.path.join(preprocessed_data_path, "dataset_splits.json")
print(f"Saving dataset splits to {splits_file}")
with open(splits_file, "w") as f:
   json.dump(splits_indices, f)

print("Data preparation complete.")
print(f"Training examples: {len(splits_indices['train_indices'])}")
print(f"Validation examples: {len(splits_indices['val_indices'])}")
print(f"Test examples: {len(splits_indices['test_indices'])}")

# Memory cleanup
import gc
gc.collect()

Loading all tokenized data...


Loading train chunks: 100%|██████████| 58/58 [00:18<00:00,  3.15it/s]


Loaded 58000 examples for train split


Loading val chunks: 100%|██████████| 1/1 [00:00<00:00,  1.23it/s]


Loaded 1000 examples for val split


Loading test chunks: 100%|██████████| 1/1 [00:00<00:00,  3.37it/s]


Loaded 1000 examples for test split
Total loaded examples: 60000
Saving combined tokenized data to /content/drive/Shareddrives/DATA266_Project/Data/sql_data/preprocessed_data/tokenized_combined_data.pt
Saving dataset splits to /content/drive/Shareddrives/DATA266_Project/Data/sql_data/preprocessed_data/dataset_splits.json
Data preparation complete.
Training examples: 58000
Validation examples: 1000
Test examples: 1000


180046

In [ ]:
import os
import json
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding works correctly

# Define a dataset class that loads data from chunks
class ChunkedSQLDataset(Dataset):
   def __init__(self, data_dir, tokenizer, max_length=512):
       self.data_dir = data_dir
       self.tokenizer = tokenizer
       self.max_length = max_length

       # Load the manifest file
       with open(os.path.join(data_dir, "manifest.json"), 'r') as f:
           self.manifest = json.load(f)

       self.total_examples = self.manifest["total_examples"]
       self.chunks = self.manifest["chunks"]
       self.chunk_size = self.manifest["chunk_size"]

       # Create a mapping of example index to chunk index and position
       self.example_map = {}
       for i in range(self.total_examples):
           chunk_idx = i // self.chunk_size
           position_in_chunk = i % self.chunk_size
           self.example_map[i] = (chunk_idx, position_in_chunk)

       # Cache for loaded chunks to improve efficiency
       self.chunk_cache = {}
       self.current_cached_chunks = []
       self.max_cached_chunks = 2  # Adjust based on memory constraints

   def __len__(self):
       return self.total_examples

   def load_chunk(self, chunk_idx):
       #Load a chunk into memory if not already cached
       if chunk_idx in self.chunk_cache:
           return self.chunk_cache[chunk_idx]

       # Load the chunk
       chunk_file = os.path.join(self.data_dir, f"training_data_chunk_{chunk_idx}.json")
       with open(chunk_file, 'r') as f:
           chunk_data = json.load(f)

       # Update cache
       self.chunk_cache[chunk_idx] = chunk_data
       self.current_cached_chunks.append(chunk_idx)

       # If cache is full, remove the oldest chunk
       if len(self.current_cached_chunks) > self.max_cached_chunks:
           oldest_chunk = self.current_cached_chunks.pop(0)
           if oldest_chunk in self.chunk_cache:
               del self.chunk_cache[oldest_chunk]

       return chunk_data

   def __getitem__(self, idx):
       # Get the chunk index and position
       chunk_idx, position_in_chunk = self.example_map[idx]

       # Load the chunk if necessary
       chunk_data = self.load_chunk(chunk_idx)

       # Get the example
       sample = chunk_data[position_in_chunk]

       # Combine prompt and response
       full_text = sample["prompt"] + sample["response"]

       # Tokenize with proper padding and truncation
       tokenized = self.tokenizer(
           full_text,
           truncation=True,
           padding="max_length",
           max_length=self.max_length,
           return_tensors="pt"
       )

       # For causal LM, labels are the same as input_ids
       tokenized["labels"] = tokenized["input_ids"].clone()

       return {key: val.squeeze(0) for key, val in tokenized.items()}

# Create the dataset
print("Setting up dataset...")
full_dataset = ChunkedSQLDataset(data_dir="sql_data", tokenizer=tokenizer)

# Create train/val/test split using indices
dataset_size = len(full_dataset)
test_size = min(1000, int(dataset_size * 0.05))
val_size = min(1000, int(dataset_size * 0.05))
train_size = dataset_size - val_size - test_size

# Use PyTorch's random_split to create the splits
train_dataset, val_dataset, test_dataset = random_split(
   full_dataset,
   [train_size, val_size, test_size],
   generator=torch.Generator().manual_seed(42)  # For reproducibility
)

print(f"Total dataset size: {dataset_size}")
print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Create data loaders
train_dataloader = DataLoader(
   train_dataset,
   batch_size=8,
   shuffle=True,
   num_workers=2
)

val_dataloader = DataLoader(
   val_dataset,
   batch_size=8,
   shuffle=False,
   num_workers=2
)

test_dataloader = DataLoader(
   test_dataset,
   batch_size=8,
   shuffle=False,
   num_workers=2
)

Setting up dataset...
Total dataset size: 60000
Training dataset size: 58000
Validation dataset size: 1000
Test dataset size: 1000


In [ ]:
class SQLDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            "input_ids": self.data[idx]["input_ids"],
            "attention_mask": self.data[idx]["attention_mask"],
            "labels": self.data[idx]["labels"]
        }

# Create train and validation splits
val_size = min(1000, int(len(tokenized_dataset) * 0.05))
train_data = tokenized_dataset[:-val_size]
val_data = tokenized_dataset[-val_size:]

train_dataset = SQLDataset(train_data)
val_dataset = SQLDataset(val_data)

print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

Training dataset size: 39000
Validation dataset size: 1000


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, LoraConfig, get_peft_model, TaskType

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer.pad_token = tokenizer.eos_token

# Load base model without device mapping
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    torch_dtype=torch.float16,
    # Don't use device_map here
)

# Move to CUDA
base_model = base_model.cuda()

# Create fresh LoRA configuration
peft_config = LoraConfig(
    r=16,

    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Apply LoRA
model = get_peft_model(base_model, peft_config)

# Load the weights
from safetensors.torch import load_file
import os

checkpoint_path = "/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_spider_lora/checkpoint-1311"
safetensors_path = os.path.join(checkpoint_path, "adapter_model.safetensors")

state_dict = load_file(safetensors_path)
model.load_state_dict(state_dict, strict=False)

# CRITICAL: Make sure parameters are set to require gradients
for name, param in model.named_parameters():
    if 'lora' in name:  # Only LoRA parameters should be trained
        param.requires_grad = True
        print(f"Enabled gradients for {name}")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Enabled gradients for base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
Enabled gradients for base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight
Enabled gradients for base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight
Enabled gradients for base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight
Enabled gradients for base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight
Enabled gradients for base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight
Enabled gradients for base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight
Enabled gradients for base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight
Enabled gradients for base_model.model.model.layers.2.self_attn.q_proj.lora_A.default.weight
Enabled gradients for base_model.model.model.layers.2.self_attn.q_proj.lora_B.default.weight
Enabled gradients for base_model.model.model.layers.2.self_attn.v_proj

In [ ]:
# Set up training arguments
from transformers import TrainingArguments

# Define output directory
output_dir = "/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_complex_sql_continued"

# Define training arguments optimized for A100
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,      # Balance between speed and memory
    per_device_eval_batch_size=4,       # Evaluation batch size
    gradient_accumulation_steps=8,      # Effective batch size = 4 x 8 = 32
    num_train_epochs=2,                 # Two epochs should be sufficient
    learning_rate=5e-5,                 # Lower learning rate for continuing training
    lr_scheduler_type="cosine",         # Cosine schedule with warmup
    warmup_ratio=0.1,                   # Longer warmup phase
    weight_decay=0.01,                  # Regularization
    fp16=True,                          # Mixed precision for speed
    gradient_checkpointing=True,        # Memory optimization
    logging_steps=10,                   # Log frequently
    save_strategy="steps",              # Save periodically
    save_steps=500,                     # Save every 500 steps
    # Removed evaluation_strategy and eval_steps
    load_best_model_at_end=False,       # Changed to False since we're not evaluating
    # Removed metric_for_best_model
    # Removed greater_is_better
    report_to="none",                   # Disable reporting
    remove_unused_columns=False,        # Keep all columns
    optim="adamw_torch",                # Use AdamW optimizer
    max_grad_norm=0.3,                  # Gradient clipping for stability
)

# If you still want evaluation, check if your version supports these parameters
try:
    training_args.eval_strategy = "steps"
    training_args.eval_steps = 500
    print("Set evaluation parameters")
except:
    print("Error")

Set evaluation parameters


In [ ]:
# Manual training loop approach
import torch
from torch.utils.data import DataLoader

# Create a simple dataloader with a smaller batch size
train_dataloader = DataLoader(
    train_dataset,
    batch_size=1,  # Start with batch size 1 for debugging
    shuffle=True
)

# Set up optimizer just for the LoRA parameters
optimizer = torch.optim.AdamW(
    [p for p in model.parameters() if p.requires_grad],
    lr=2e-5,
    weight_decay=0.01
)

# Put model in training mode
model.train()

# Simple training loop
print("Starting manual training loop")
for epoch in range(2):  # 2 epochs
    running_loss = 0.0
    for step, batch in enumerate(train_dataloader):
        # Move batch to GPU
        batch = {k: v.cuda() for k, v in batch.items()}

        # Forward pass with explicit gradient tracking
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"],
            use_cache=False  # Disable cache to avoid issues
        )

        loss = outputs.loss

        # Check if loss has gradients
        if loss.requires_grad:
            print(f"Step {step}: Loss requires grad: {loss.requires_grad}")
        else:
            print(f"Step {step}: Loss does NOT require grad!")

        # Manual backward pass
        loss.backward()

        # Check if any gradients were actually computed
        has_grad = False
        for name, param in model.named_parameters():
            if param.requires_grad and param.grad is not None:
                has_grad = True
                break

        if has_grad:
            print(f"Step {step}: Gradients computed successfully")
        else:
            print(f"Step {step}: No gradients were computed!")

        # Update weights
        optimizer.step()
        optimizer.zero_grad()

        # Track loss
        running_loss += loss.item()

        # Log progress
        if step % 10 == 0:
            print(f"Epoch {epoch}, Step {step}, Loss {loss.item():.4f}, Avg Loss: {running_loss/(step+1):.4f}")

        # Save checkpoint periodically
        if step % 500 == 0 and step > 0:
            checkpoint_dir = f"/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_complex_sql_continued/manual-epoch-{epoch}-step-{step}"
            model.save_pretrained(checkpoint_dir)
            tokenizer.save_pretrained(checkpoint_dir)
            print(f"Saved checkpoint to {checkpoint_dir}")

    # Save at end of epoch
    epoch_dir = f"/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_complex_sql_continued/manual-epoch-{epoch}"
    model.save_pretrained(epoch_dir)
    tokenizer.save_pretrained(epoch_dir)
    print(f"Epoch {epoch} completed. Average loss: {running_loss / len(train_dataloader):.4f}")

# Save final model
final_dir = f"/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_complex_sql_continued/spider_x_synsql_model"
model.save_pretrained(final_dir)
tokenizer.save_pretrained(final_dir)
print(f"Training complete! Final model saved to {final_dir}")

In [ ]:
# Merge the adapter into the base model
print("Merging adapter into base model...")
merged_model = model.merge_and_unload()

# Save the merged model
merged_model_dir = "/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_merged_ready"
merged_model.save_pretrained(merged_model_dir)
tokenizer.save_pretrained(merged_model_dir)
print(f"Saved merged model to {merged_model_dir}")

# Load the merged model fresh
from transformers import AutoModelForCausalLM
import torch
from peft import LoraConfig, get_peft_model, TaskType

print("Loading merged model...")
fresh_model = AutoModelForCausalLM.from_pretrained(
    merged_model_dir,
    torch_dtype=torch.float16,
)
fresh_model = fresh_model.cuda()

# Apply a new LoRA adapter with more targets
new_peft_config = LoraConfig(
    r=32,                          # Higher rank
    lora_alpha=64,                 # Scaled with rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # All attention components
    lora_dropout=0.05,
    bias="none",                   # No bias training
    task_type=TaskType.CAUSAL_LM
)

# Create new LoRA model
new_model = get_peft_model(fresh_model, new_peft_config)
print("Created fresh model with new LoRA configuration")
new_model.print_trainable_parameters()

# Create a simple dataloader with a small batch size
from torch.utils.data import DataLoader
train_dataloader = DataLoader(
    train_dataset,
    batch_size=1,  # Start with batch size 1
    shuffle=True
)

# Set up optimizer just for the LoRA parameters
optimizer = torch.optim.AdamW(
    [p for p in new_model.parameters() if p.requires_grad],
    lr=2e-5,
    weight_decay=0.01
)

# Simple training loop
print("Starting training with merged model + fresh LoRA")
new_model.train()

for epoch in range(2):  # 2 epochs
    running_loss = 0.0
    for step, batch in enumerate(train_dataloader):
        # Move batch to GPU
        batch = {k: v.cuda() for k, v in batch.items()}

        # Forward pass
        outputs = new_model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"],
            use_cache=False  # Disable cache
        )

        loss = outputs.loss
        print(f"Step {step}: Loss: {loss.item():.4f}, requires_grad: {loss.requires_grad}")

        # Backward pass
        loss.backward()

        # Check if gradients were computed
        has_grad = any(p.grad is not None for p in new_model.parameters() if p.requires_grad)
        if has_grad:
            print(f"Step {step}: Gradients computed successfully")
        else:
            print(f"Step {step}: No gradients computed!")
            continue  # Skip this step if no gradients

        # Update weights
        optimizer.step()
        optimizer.zero_grad()

        # Track loss
        running_loss += loss.item()

        # Log progress
        if step % 10 == 0:
            print(f"Epoch {epoch}, Step {step}, Loss {loss.item():.4f}, Avg Loss: {running_loss/(step+1):.4f}")

        # Save checkpoint periodically
        if step % 500 == 0 and step > 0:
            checkpoint_dir = f"/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_complex_sql_continued/manual-epoch-{epoch}-step-{step}"
            new_model.save_pretrained(checkpoint_dir)
            print(f"Saved checkpoint to {checkpoint_dir}")

    # Save at end of epoch
    epoch_dir = f"/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_complex_sql_continued/manual-epoch-{epoch}"
    new_model.save_pretrained(epoch_dir)
    print(f"Epoch {epoch} completed. Average loss: {running_loss / len(train_dataloader):.4f}")

# Save final model
final_dir = f"/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_complex_sql_continued/spider_x_synsql_model"
new_model.save_pretrained(final_dir)
tokenizer.save_pretrained(final_dir)
print(f"Training complete! Final model saved to {final_dir}")

Merging adapter into base model...
Saved merged model to /content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_merged_ready
Loading merged model...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Step 6593: Gradients computed successfully
Step 6594: Loss: 0.1017, requires_grad: True
Step 6594: Gradients computed successfully
Step 6595: Loss: 0.2723, requires_grad: True
Step 6595: Gradients computed successfully
Step 6596: Loss: 0.3878, requires_grad: True
Step 6596: Gradients computed successfully
Step 6597: Loss: 0.3190, requires_grad: True
Step 6597: Gradients computed successfully
Step 6598: Loss: 0.2701, requires_grad: True
Step 6598: Gradients computed successfully
Step 6599: Loss: 0.1377, requires_grad: True
Step 6599: Gradients computed successfully
Step 6600: Loss: 0.1214, requires_grad: True
Step 6600: Gradients computed successfully
Epoch 0, Step 6600, Loss 0.1214, Avg Loss: 0.2576
Step 6601: Loss: 0.2008, requires_grad: True
Step 6601: Gradients computed successfully
Step 6602: Loss: 0.1291, requires_grad: True
Step 6602: Gradients computed successfully
Step 6603: Loss: 0.3323, requires_grad: True
Step 6603: Gradient

KeyboardInterrupt: 

Our Attempted Approach
In our recent attempts, we were:

Starting with an already merged model (where the adapter weights had been merged into the base model)
Creating a new LoRA adapter on top of this merged model
Trying to train the new adapter

The Critical Difference
The merged model approach introduces complexity:

When you merge a LoRA adapter into a base model, the adapter weights become "baked in" to the base model weights
This creates a new starting point that has already incorporated the adapter's changes
Then adding a new LoRA adapter on top of this merged model creates a different training dynamic

In contrast, the working approach:

Keeps a clean separation between base model and adapter
Directly loads and continues training with the existing adapter
Maintains the original adapter structure and training pattern

Why This Matters

Gradient Flow: The working approach maintains the original gradient flow paths established in the initial training
Parameter Structure: It preserves the exact parameter structure that was successfully used before
Continuity: It's truly "continuing" training rather than starting fresh on top of a merged model

In [ ]:
import os
import torch
import gc
import json
import random
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, LoraConfig, get_peft_model, TaskType
from safetensors.torch import load_file
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

# Set random seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

# Set paths directly without a base path
checkpoint_path = "Checkpoints/mistral_spider_lora/checkpoint-1311"
output_dir = "Checkpoints/mistral_complex_sql_training_4090"
preprocessed_data_path = "data/preprocessed_data"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Clean up GPU memory
gc.collect()
torch.cuda.empty_cache()

print("Setup complete. Using GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

Setup complete. Using GPU: NVIDIA GeForce RTX 4090


In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding works correctly

# Determine if BF16 is supported
def is_bf16_supported():
    if torch.cuda.is_available():
        compute_capability = torch.cuda.get_device_capability()[0]
        return compute_capability >= 8  # Ampere (RTX 30xx) and newer support BF16
    return False

has_bf16 = is_bf16_supported()
print(f"BF16 precision support: {has_bf16}")

# Load base model without device mapping
print("Loading base Mistral-7B model...")
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    torch_dtype=torch.bfloat16 if has_bf16 else torch.float16,
    # Don't use device_map here
)

# Move to CUDA explicitly
base_model = base_model.cuda()
print("Base model loaded and moved to CUDA")

# Create LoRA configuration matching your previous training
peft_config = LoraConfig(
    r=16,                      # Match original adapter config
    lora_alpha=32,             # Match original adapter config
    target_modules=["q_proj", "v_proj"],  # Match original target modules
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Apply LoRA
model = get_peft_model(base_model, peft_config)
print("Created LoRA model with configuration matching previous training")

# Load adapter weights from checkpoint
safetensors_path = os.path.join(checkpoint_path, "adapter_model.safetensors")
if os.path.exists(safetensors_path):
    print(f"Loading adapter weights from {safetensors_path}")
    state_dict = load_file(safetensors_path)
    model.load_state_dict(state_dict, strict=False)
    print("Adapter weights loaded successfully")
else:
    print(f"WARNING: Could not find adapter weights at {safetensors_path}")

# CRITICAL: Make sure parameters are set to require gradients
for name, param in model.named_parameters():
    if 'lora' in name:  # Only LoRA parameters should be trained
        param.requires_grad = True
        print(f"Enabled gradients for {name}")

# Verify trainable parameters
trainable_params = [p for p in model.parameters() if p.requires_grad]
print(f"Number of trainable parameters: {len(trainable_params)}")
print(f"Total trainable parameter count: {sum(p.numel() for p in trainable_params)}")

# Print model structure info
model.print_trainable_parameters()

BF16 precision support: True
Loading base Mistral-7B model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Base model loaded and moved to CUDA
Created LoRA model with configuration matching previous training
Loading adapter weights from Checkpoints/mistral_spider_lora/checkpoint-1311/adapter_model.safetensors
Adapter weights loaded successfully
Enabled gradients for base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
Enabled gradients for base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight
Enabled gradients for base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight
Enabled gradients for base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight
Enabled gradients for base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight
Enabled gradients for base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight
Enabled gradients for base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight
Enabled gradients for base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight
Enabled gradient

In [ ]:
# Custom dataset class
class SQLDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            "input_ids": self.data[idx]["input_ids"],
            "attention_mask": self.data[idx]["attention_mask"],
            "labels": self.data[idx]["labels"]
        }

# Load the tokenized data and splits
tokenized_file = os.path.join(preprocessed_data_path, "tokenized_combined_data.pt")
splits_file = os.path.join(preprocessed_data_path, "dataset_splits.json")

print(f"Loading tokenized data from {tokenized_file}")
tokenized_data = torch.load(tokenized_file)
print(f"Loaded {len(tokenized_data)} tokenized examples")

print(f"Loading dataset splits from {splits_file}")
with open(splits_file, 'r') as f:
    splits = json.load(f)

# Create the datasets based on the saved splits
train_indices = splits["train_indices"]
val_indices = splits["val_indices"]

print(f"Creating datasets with {len(train_indices)} training examples and {len(val_indices)} validation examples")
train_data = [tokenized_data[i] for i in train_indices]
val_data = [tokenized_data[i] for i in val_indices]

train_dataset = SQLDataset(train_data)
val_dataset = SQLDataset(val_data)

print(f"Created training dataset with {len(train_dataset)} examples")
print(f"Created validation dataset with {len(val_dataset)} examples")

# Free up memory
del tokenized_data
gc.collect()
torch.cuda.empty_cache()

# Create DataLoader for training
batch_size = 1  # Start with a small batch size for stability
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0  # No parallel workers to avoid memory issues
)

# Create DataLoader for validation (optional)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0
)

print(f"Created DataLoader with batch size {batch_size}")
print(f"Training steps per epoch: {len(train_dataloader)}")

Loading tokenized data from data/preprocessed_data/tokenized_combined_data.pt
Loaded 37000 tokenized examples
Loading dataset splits from data/preprocessed_data/dataset_splits.json
Creating datasets with 35150 training examples and 1850 validation examples
Created training dataset with 35150 examples
Created validation dataset with 1850 examples
Created DataLoader with batch size 1
Training steps per epoch: 35150


In [ ]:
# Training hyperparameters
learning_rate = 2e-5
weight_decay = 0.01
num_epochs = 2
grad_accumulation_steps = 8  # Accumulate gradients over multiple steps

# Create optimizer for LoRA parameters only
optimizer = torch.optim.AdamW(
    [p for p in model.parameters() if p.requires_grad],
    lr=learning_rate,
    weight_decay=weight_decay,
    betas=(0.9, 0.999)
)

# Create a simple linear learning rate scheduler with warmup
from transformers import get_scheduler
num_training_steps = len(train_dataloader) * num_epochs // grad_accumulation_steps
num_warmup_steps = int(num_training_steps * 0.03)  # 3% warmup

# Create scheduler
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

print("Training configuration:")
print(f"- Learning rate: {learning_rate}")
print(f"- Weight decay: {weight_decay}")
print(f"- Epochs: {num_epochs}")
print(f"- Gradient accumulation steps: {grad_accumulation_steps}")
print(f"- Effective batch size: {batch_size * grad_accumulation_steps}")
print(f"- Total training steps: {num_training_steps}")
print(f"- Warmup steps: {num_warmup_steps}")

Training configuration:
- Learning rate: 2e-05
- Weight decay: 0.01
- Epochs: 2
- Gradient accumulation steps: 8
- Effective batch size: 8
- Total training steps: 8787
- Warmup steps: 263


In [ ]:
# Set model to training mode
model.train()

# Training loop
print("\n Starting training ")
global_step = 0
total_loss = 0

# Progress tracking
from tqdm.auto import tqdm

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    epoch_loss = 0
    optimizer.zero_grad()  # Reset gradients at start of epoch

    # Progress bar for this epoch
    progress_bar = tqdm(total=len(train_dataloader), desc=f"Epoch {epoch+1}")

    for step, batch in enumerate(train_dataloader):
        # Move batch to GPU
        batch = {k: v.cuda() for k, v in batch.items()}

        # Forward pass
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"],
            use_cache=False  # Disable KV cache for training
        )

        # Get loss and scale for gradient accumulation
        loss = outputs.loss / grad_accumulation_steps

        # Log the loss value
        if step % 10 == 0:
            print(f"Step {step}: Loss: {loss.item() * grad_accumulation_steps:.4f}, requires_grad: {loss.requires_grad}")

        # Backward pass
        loss.backward()

        # Check for gradient computation
        has_grad = any(p.grad is not None for p in model.parameters() if p.requires_grad)
        if step % 50 == 0:
            if has_grad:
                print(f"Step {step}: Gradients computed successfully")
            else:
                print(f"Step {step}: WARNING - No gradients computed!")

        # Update weights after accumulating gradients
        if (step + 1) % grad_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            # Clip gradients to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(
                [p for p in model.parameters() if p.requires_grad],
                max_norm=1.0
            )

            # Update weights
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

            # Increment global step
            global_step += 1

            # Save checkpoint periodically
            if global_step % 100 == 0:
                checkpoint_dir = os.path.join(output_dir, f"checkpoint-{global_step}")
                model.save_pretrained(checkpoint_dir)
                print(f"Saved checkpoint to {checkpoint_dir}")

        # Update tracking metrics
        epoch_loss += loss.item() * grad_accumulation_steps
        total_loss += loss.item() * grad_accumulation_steps

        # Update progress bar
        progress_bar.update(1)
        progress_bar.set_postfix({"loss": epoch_loss / (step + 1)})

        # Clear CUDA cache periodically
        if step % 500 == 0 and step > 0:
            gc.collect()
            torch.cuda.empty_cache()

    # End of epoch
    progress_bar.close()
    avg_epoch_loss = epoch_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} completed. Average loss: {avg_epoch_loss:.4f}")

    # Save model at end of epoch
    epoch_dir = os.path.join(output_dir, f"epoch-{epoch+1}")
    model.save_pretrained(epoch_dir)
    tokenizer.save_pretrained(epoch_dir)
    print(f"Saved model at end of epoch {epoch+1} to {epoch_dir}")

# End of training
print(f"\nTraining completed. Average loss: {total_loss / (len(train_dataloader) * num_epochs):.4f}")


 Starting training 

Epoch 1/2


Epoch 1:   0%|          | 0/35150 [00:00<?, ?it/s]

Step 0: Loss: 0.3508, requires_grad: True
Step 0: Gradients computed successfully
Step 10: Loss: 0.4638, requires_grad: True
Step 20: Loss: 0.3787, requires_grad: True
Step 30: Loss: 0.5463, requires_grad: True
Step 40: Loss: 0.4859, requires_grad: True
Step 50: Loss: 0.5323, requires_grad: True
Step 50: Gradients computed successfully
Step 60: Loss: 0.4579, requires_grad: True
Step 70: Loss: 0.5387, requires_grad: True
Step 80: Loss: 0.5212, requires_grad: True
Step 90: Loss: 0.5180, requires_grad: True
Step 100: Loss: 0.4386, requires_grad: True
Step 100: Gradients computed successfully
Step 110: Loss: 0.5440, requires_grad: True
Step 120: Loss: 0.6301, requires_grad: True
Step 130: Loss: 0.4099, requires_grad: True
Step 140: Loss: 0.4736, requires_grad: True
Step 150: Loss: 0.5047, requires_grad: True
Step 150: Gradients computed successfully
Step 160: Loss: 0.4262, requires_grad: True
Step 170: Loss: 0.6133, requires_grad: True
Step 180: Loss: 0.5972, requires_grad: True
Step 190: 

Epoch 2:   0%|          | 0/35150 [00:00<?, ?it/s]

Step 0: Loss: 0.2351, requires_grad: True
Step 0: Gradients computed successfully
Step 10: Loss: 0.4300, requires_grad: True
Step 20: Loss: 0.2056, requires_grad: True
Step 30: Loss: 0.1964, requires_grad: True
Step 40: Loss: 0.1094, requires_grad: True
Saved checkpoint to Checkpoints/mistral_complex_sql_training_4090/checkpoint-4400
Step 50: Loss: 0.1670, requires_grad: True
Step 50: Gradients computed successfully
Step 60: Loss: 0.3024, requires_grad: True
Step 70: Loss: 0.3094, requires_grad: True
Step 80: Loss: 0.1763, requires_grad: True
Step 90: Loss: 0.2580, requires_grad: True
Step 100: Loss: 0.2136, requires_grad: True
Step 100: Gradients computed successfully
Step 110: Loss: 0.1623, requires_grad: True
Step 120: Loss: 0.2967, requires_grad: True
Step 130: Loss: 0.3950, requires_grad: True
Step 140: Loss: 0.3261, requires_grad: True
Step 150: Loss: 0.1117, requires_grad: True
Step 150: Gradients computed successfully
Step 160: Loss: 0.2084, requires_grad: True
Step 170: Loss: 